# what's the neuron yield across probes, experimenters and recording sites?
Anne Urai & Nate Miska, 2020

In [1]:
# GENERAL THINGS FOR COMPUTING AND PLOTTING
import pandas as pd
import numpy as np
import os, sys, time
import scipy as sp

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# ibl specific things
import datajoint as dj
from ibl_pipeline import reference, subject, action, acquisition, data, behavior
from ibl_pipeline.analyses import behavior as behavior_analysis
ephys = dj.create_virtual_module('ephys', 'ibl_ephys')
figpath = os.path.join(os.path.expanduser('~'), 'Data/Figures_IBL')

Connecting anneurai@datajoint.internationalbrainlab.org:3306


## 1. neuron yield per lab and Npix probe over time
Replicates https://github.com/int-brain-lab/analysis/blob/master/python/probe_performance_over_sessions.py using DJ

In [2]:
probe_insertions = ephys.ProbeInsertion * ephys.DefaultCluster.Metrics * subject.SubjectLab \
                    * (acquisition.SessionProject
                      & 'session_project = "ibl_neuropixel_brainwide_01"') \
                    * behavior_analysis.SessionTrainingStatus
probe_insertions = probe_insertions.proj('probe_serial_number', 'probe_model_name', 'lab_name', 'metrics',
                                         'good_enough_for_brainwide_map',
                                         session_date='DATE(session_start_time)')
clusts = probe_insertions.fetch(format='frame').reset_index()

DataJointError: Attribute `probe_serial_number` is not found

In [ ]:
# put metrics into df columns from the blob (feature request: can these be added as attributes instead?)
for kix, k in enumerate(['ks2_label']):
    tmp_var = []
    for id, c in clusts.iterrows():
        if k in c['metrics'].keys():
            tmp = c['metrics'][k]
        else:
            tmp = np.nan
        tmp_var.append(tmp)
    clusts[k] = tmp_var

In [ ]:
# hofer and mrsic-flogel probes are shared
clusts['lab_name'] = clusts['lab_name'].str.replace('mrsicflogellab','swclab')
clusts['lab_name'] = clusts['lab_name'].str.replace('hoferlab','swclab')
clusts.lab_name.unique()

In [ ]:
clusts['probe_name'] = clusts['lab_name'] + ', ' + clusts['probe_model_name'] + ': ' + clusts['probe_serial_number']
clusts_summ = clusts.groupby(['lab_name', 'probe_name', 'session_start_time', 'ks2_label'])['session_date'].count().reset_index()

# use recording session number instead of date
clusts_summ['recording'] = clusts_summ.groupby(['probe_name']).cumcount() + 1

In [ ]:
sns.set(style="ticks", context="paper")
g, axes = plt.subplots(6,6,figsize=(18,20))

for probe, ax in zip(clusts_summ.probe_name.unique(), axes.flatten()):
    df = clusts_summ[clusts_summ.probe_name==probe].groupby(['session_start_time','ks2_label']).session_date.sum()
    df.unstack().plot.barh(ax=ax, stacked=True, legend=False, colormap='Pastel2')
    ax.set_title(probe, fontsize=12)
    ax.axvline(x=60, color='seagreen', linestyle="--")
    ax.set_yticks([])
    ax.set_ylabel('')
    ax.set_ylim([-1, np.max([max(ax.get_ylim()), 10])])
    ax.set_xlim([0, 1000])
    
axes.flatten()[-1].set_axis_off()
sns.despine(trim=True)   
plt.tight_layout()
plt.xlabel('Number of KS2 neurons')
plt.ylabel('Recording session')
g.savefig(os.path.join(figpath, 'probe_yield_oversessions.pdf'))

# 2. what is the overall yield of sessions, neurons etc?

In [ ]:
## overall distribution of neurons per session
g = sns.FacetGrid(data=clusts_summ, hue='ks2_label', palette='Set2')
g.map(sns.distplot, "session_date", bins=np.arange(10, 500, 15), hist=True, rug=False, kde=False).add_legend()
for ax in g.axes.flatten():
    ax.axvline(x=60, color='seagreen', linestyle="--")
    
g.set_xlabels('Number of KS2 neurons')
g.set_ylabels('Number of sessions')
g.savefig(os.path.join(figpath, 'probe_yield_allrecs.pdf'))

print('TOTAL YIELD SO FAR:')
clusts.groupby(['ks2_label'])['ks2_label'].count()

In [ ]:
## overall distribution of neurons per session
g = sns.FacetGrid(data=clusts_summ, hue='ks2_label', col_wrap=4, col='lab_name', palette='Set2')
g.map(sns.distplot, "session_date", bins=np.arange(10, 500, 15), hist=True, rug=False, kde=False).add_legend()
for ax in g.axes.flatten():
    ax.axvline(x=60, color='seagreen', linestyle="--")
    
g.set_xlabels('Number of KS2 neurons')
g.set_ylabels('Number of sessions')
#g.savefig(os.path.join(figpath, 'probe_yield_allrecs_perlab.pdf'))


In [ ]:
## overall number of sessions that meet criteria for behavior and neural yield
sessions = clusts.loc[clusts.ks2_label == 'good', :].groupby(['lab_name', 'subject_uuid', 'session_start_time', 
                           'good_enough_for_brainwide_map'])['cluster_id'].count().reset_index()
sessions['enough_neurons'] = (sessions['cluster_id'] > 60)
ct = sessions.groupby(['good_enough_for_brainwide_map', 'enough_neurons'])['cluster_id'].count().reset_index()
print('total nr of sessions: %d'%ct.cluster_id.sum())
pd.pivot_table(ct, columns=['good_enough_for_brainwide_map'], values=['cluster_id'], index=['enough_neurons'])
#sessions.describe()
# pd.pivot_table(df, values='cluster_id', index=['lab_name'],
#                     columns=['enough_neurons'], aggfunc=np.sum)

In [ ]:
# check that this pandas wrangling is correct...
ephys_sessions = acquisition.Session * subject.Subject * subject.SubjectLab \
                    * (acquisition.SessionProject
                      & 'session_project = "ibl_neuropixel_brainwide_01"') \
                    * behavior_analysis.SessionTrainingStatus \
                    & ephys.ProbeInsertion & ephys.DefaultCluster.Metrics 
ephys_sessions = ephys_sessions.fetch(format='frame').reset_index()
# ephys_sessions
# ephys_sessions.groupby(['good_enough_for_brainwide_map'])['session_start_time'].describe()

In [ ]:
# which sessions do *not* show good enough behavior?
ephys_sessions.loc[ephys_sessions.good_enough_for_brainwide_map == 0, :].groupby([
                        'lab_name', 'subject_nickname', 'session_start_time'])['session_start_time'].unique()

In [ ]:
# per lab, what's the drop-out due to behavior? 
ephys_sessions['good_enough_for_brainwide_map'] = ephys_sessions['good_enough_for_brainwide_map'].astype(int)
ephys_sessions.groupby(['lab_name'])['good_enough_for_brainwide_map'].describe()

In [ ]:
ephys_sessions['good_enough_for_brainwide_map'].describe()

In [ ]:
# per lab, what's the dropout due to yield?
sessions['enough_neurons'] = sessions['enough_neurons'].astype(int)
sessions.groupby(['lab_name'])['enough_neurons'].describe()

In [ ]:
## also show the total number of neurons, only from good behavior sessions
probe_insertions = ephys.ProbeInsertion * ephys.DefaultCluster.Metrics * subject.SubjectLab \
                    * (acquisition.SessionProject
                      & 'session_project = "ibl_neuropixel_brainwide_01"') \
                    * (behavior_analysis.SessionTrainingStatus & 
                       'good_enough_for_brainwide_map = 1')
probe_insertions = probe_insertions.proj('probe_serial_number', 'probe_model_name', 'lab_name', 'metrics',
                                         'good_enough_for_brainwide_map',
                                         session_date='DATE(session_start_time)')
clusts = probe_insertions.fetch(format='frame').reset_index()

# put metrics into df columns from the blob (feature request: can these be added as attributes instead?)
for kix, k in enumerate(['ks2_label']):
    tmp_var = []
    for id, c in clusts.iterrows():
        if k in c['metrics'].keys():
            tmp = c['metrics'][k]
        else:
            tmp = np.nan
        tmp_var.append(tmp)
    clusts[k] = tmp_var
    
# hofer and mrsic-flogel probes are shared
clusts['lab_name'] = clusts['lab_name'].str.replace('mrsicflogellab','swclab')
clusts['lab_name'] = clusts['lab_name'].str.replace('hoferlab','swclab')
clusts.lab_name.unique()

clusts['probe_name'] = clusts['lab_name'] + ', ' + clusts['probe_model_name'] + ': ' + clusts['probe_serial_number']
clusts_summ = clusts.groupby(['lab_name', 'probe_name', 'session_start_time', 'ks2_label'])['session_date'].count().reset_index()

# use recording session number instead of date
clusts_summ['recording'] = clusts_summ.groupby(['probe_name']).cumcount() + 1

## overall distribution of neurons per session
g = sns.FacetGrid(data=clusts_summ, hue='ks2_label', palette='Set2')
g.map(sns.distplot, "session_date", bins=np.arange(10, 500, 15), hist=True, rug=False, kde=False).add_legend()
for ax in g.axes.flatten():
    ax.axvline(x=60, color='seagreen', linestyle="--")
    
g.set_xlabels('Number of KS2 neurons')
g.set_ylabels('Number of sessions')
g.savefig(os.path.join(figpath, 'probe_yield_allrecs_goodsessions.pdf'))

print('TOTAL YIELD (from good sessions) SO FAR:')
clusts.groupby(['ks2_label'])['ks2_label'].count()

## 2. how does probe yield in the repeated site differ between mice/experimenters?

In [ ]:
probes_rs = (ephys.ProbeTrajectory & 'insertion_data_source = "Planned"'
             & 'x BETWEEN -2400 AND -2100' & 'y BETWEEN -2100 AND -1900' & 'theta BETWEEN 14 AND 16')

clust = ephys.DefaultCluster * ephys.DefaultCluster.Metrics * probes_rs * subject.SubjectLab() * subject.Subject()
clust = clust.proj('cluster_amp', 'cluster_depth', 'firing_rate', 'subject_nickname', 'lab_name','metrics',
                   'x', 'y', 'theta', 'phi', 'depth')
clusts = clust.fetch(format='frame').reset_index()
clusts['col_name'] = clusts['lab_name'] + ', ' + clusts['subject_nickname']

# put metrics into df columns from the blob
for kix, k in enumerate(clusts['metrics'][0].keys()):
    tmp_var = []
    for id, c in clusts.iterrows():
        if k in c['metrics'].keys():
            tmp = c['metrics'][k]
        else:
            tmp = np.nan
        tmp_var.append(tmp)
    clusts[k] = tmp_var

clusts

In [ ]:
sns.set(style="ticks", context="paper")
g, axes = plt.subplots(1,1,figsize=(4,4))
df = clusts.groupby(['col_name', 'ks2_label']).ks2_label.count()
df.unstack().plot.barh(ax=axes, stacked=True, legend=True, colormap='Pastel2')
axes.axvline(x=60, color='seagreen', linestyle="--")
axes.set_ylabel('')
sns.despine(trim=True)   
plt.xlabel('Number of KS2 neurons')
g.savefig(os.path.join(figpath, 'probe_yield_rs.pdf'))

In [ ]:
## firing rate as a function of depth
print('plotting')
g = sns.FacetGrid(data=clusts, col='col_name', col_wrap=4, hue='ks2_label',
                  palette='Pastel2', col_order=sorted(clusts.col_name.unique()))
g.map(sns.scatterplot, "firing_rate", "cluster_depth", alpha=0.5).add_legend()
g.set_titles('{col_name}')
g.set_xlabels('Firing rate (spks/s)')
g.set_ylabels('Depth')
plt.tight_layout()
sns.despine(trim=True)
g.savefig(os.path.join(figpath, 'neurons_rsi_firingrate.pdf'))
